# D Flip Flop

Demonstrate a D flip flop with asynchronous reset.

The flop's Q output is connected to an inverter,
which is connected back into the D input.

The resulting simulation will have the following waveform:

```
T    0  1  2  3  4  5  6  7  8  9 ...

clk  /‾‾\__/‾‾\__/‾‾\__/‾‾\__/‾‾\__/‾‾\__/‾‾\__/‾‾\__/‾‾\__/‾‾\
rst  ___/‾‾\___________________________________________________

q    XXXX________/‾‾‾‾‾\_____/‾‾‾‾‾\_____/‾‾‾‾‾\_____/‾‾‾‾‾\___
d    XXXX‾‾‾‾‾‾‾‾\_____/‾‾‾‾‾\_____/‾‾‾‾‾\_____/‾‾‾‾‾\_____/‾‾‾
```

Notice:
* `q` initial value is `X`, meaning "uninitialized".
* `q` is reset after `T=1`, the positive edge of `rst`.
* `q` does not transition at `T=2` b/c `rst=1`.
* `d` appears to update simultaneously with `q`.

In [1]:
import sys
from pathlib import Path

WORKSPACE = Path("..")

sys.path.insert(0, str(WORKSPACE / "src"))

In [2]:
from bvwx import Vec
from vcd import VCDWriter

from seqlogic import Module, run
from seqlogic.control.sync import drv_clock, drv_reset

In [3]:
class Top(Module):
    def build(self):
        clk = self.logic(name="clk", dtype=Vec[1])
        rst = self.logic(name="rst", dtype=Vec[1])

        q = self.logic(name="q", dtype=Vec[1])
        d = self.logic(name="d", dtype=Vec[1])

        # d = NOT(q)
        self.expr(d, ~q)

        # DFF w/ async positive (active high) reset, reset to 0
        self.dff(q, d, clk, rst=rst, rval="1b0")

        # Positive clock w/ no phase shift, period T=2, 50% duty cycle
        self.drv(drv_clock(clk, shiftticks=0, onticks=1, offticks=1))

        # Positive reset asserting from T=[1..2]
        self.drv(drv_reset(rst, shiftticks=1, onticks=1))

In [4]:
with (
    open("dff.vcd", "w") as f,
    VCDWriter(f, timescale="1ns") as vcdw,
):
    top = Top(name="top")
    top.dump_vcd(vcdw, ".*")
    run(top.main(), ticks=20)